In [ ]:
from __future__ import print_function

%matplotlib inline

# Tutorial: understand how FluidSim works

A goal of FluidSim is to be as simple as possible to allow anyone knowing a little bit of Python to understand how it works internally. For this tutorial, it is assumed that the reader knows how to run simulations with FluidSim. If it is not the case, first read the tutorial [running a simulation (user perspective)](tuto_user.ipynb).

## A class to organize parameters

First, we need to present the important class [fluiddyn.util.paramcontainer.ParamContainer](http://fluiddyn.readthedocs.org/en/latest/generated/fluiddyn.util.paramcontainer.html) used to contain information.

In [ ]:
from fluiddyn.util.paramcontainer import ParamContainer

params = ParamContainer(tag="params")
params._set_attribs({"a0": 1, "a1": 1})
params._set_attrib("a2", 1)
params._set_child("child0", {"a0": 1})
params.a2 = 2
params.child0.a0 = "other option"

A `ParamContainer` can be represented as xml

In [ ]:
params

FluidSim uses instances of this class to store the information of a particular solver and the parameters of a particular simulation.

## The Simul classes and the default parameters

The first step to run a simulation is to import a Simul class from a solver module, for example:

In [ ]:
from fluidsim.solvers.ns2d.solver import Simul

Any solver module has to define a class called Simul which has to have some important attributes:

In [ ]:
[name for name in dir(Simul) if not name.startswith("__")]

The first attribute `InfoSolver` is a class deriving from [InfoSolverBase](../generated/fluidsim.base.solvers.info_base.rst#fluidsim.base.solvers.info_base.InfoSolverBase) (which is a [ParamContainer](https://fluiddyn.readthedocs.io/en/latest/generated/fluiddyn.util.paramcontainer.html)). This class is usually defined in the `solver` module. It is used during the instantiation of the Simul object to produce a [ParamContainer](https://fluiddyn.readthedocs.io/en/latest/generated/fluiddyn.util.paramcontainer.html) containing a description of the solver, in practice the names and the modules of the classes used for the different tasks that need to be performed during the simulation.

There are also four other functions. `compute_freq_diss` and `tendencies_nonlin` are used during the simulation and describe the equations that are solved.

`create_default_params` and `_complete_params_with_default` are used to produce the `ParamContainer` containing the default parameters for a simulation:

In [ ]:
params = Simul.create_default_params()

During the creation of `params`, the class `InfoSolver` has been used to create a `ParamContainer` named `info_solver`:

In [ ]:
Simul.info_solver

We see that this solver uses many classes and that they are organized in tasks ("Operator", "InitFields", "TimeStepping", "State", "Output", "Forcing"). Some first-level classes (for example "Output") have second-level classes ("PrintStdOut", "Spectra", "PhysFields", etc.). Such description of a solver is very general. It is also very conveniant to create a new solver from a similar existing solver.

Every classes can have a class function or a static function `_complete_params_with_default` that is called when the object containing the default parameters is created.

The objects `params` and `Simul.info_solver` are then used to instantiate the simulation (here with the default parameters for the solver):

In [ ]:
sim = Simul(params)

Let's print the attributes of `sim` that are not class attributes:

In [ ]:
[name for name in dir(sim) if not name.startswith("_") and name not in dir(Simul)]

Except `name_run` and `info`, the attributes are instances of the first-level classes defined in `Simul.info_solver`. These different objects have to interact together. We are going to present these different hierarchies of classes but first we come back to the two functions describing the equations in a pseudo-spectral solver.

## Description of the solved equations

The functions `Simul.compute_freq_diss` and `Simul.tendencies_nonlin` define the solved equations. Looking at the documentation of the solver module [fluidsim.solvers.ns2d.solver](../generated/fluidsim.solvers.ns2d.solver.rst), we see that `Simul.tendencies_nonlin` is defined in this module and that `Simul.compute_freq_diss` is inherited from the base class [fluidsim.base.solvers.pseudo_spect.SimulBasePseudoSpectral](../generated/fluidsim.base.solvers.pseudo_spect.rst#fluidsim.base.solvers.pseudo_spect.SimulBasePseudoSpectral). By clicking on these links, you can look at the documentation and the sources of these functions. The documentation explains how this function define the solved equations. I think the sources are quite clear and can be understood by anyone knowing a little bit of Python for science. Most of the objects involved in these functions are functions or [numpy.ndarray](http://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.html).

## State classes (`sim.state`)

`sim.state` is an instance of [fluidsim.solvers.ns2d.state.StateNS2D](../generated/fluidsim.solvers.ns2d.state.rst#fluidsim.solvers.ns2d.state.StateNS2D). It contains `numpy.ndarray`, actually slightly modified `numpy.ndarray` named [fluidsim.base.setofvariables.SetOfVariables](../generated/fluidsim.base.setofvariables.rst#fluidsim.base.setofvariables.SetOfVariables). This class is used to stack variables together in a single `numpy.ndarray`.

The state classes are also able to compute other variables from the state of the simulation. It is an interface hiding the actual way the data are stored.

## Operator classes (`sim.oper`)

`sim.oper` is an instance of [fluidsim.operators.operators2d.OperatorsPseudoSpectral2D](../generated/fluidsim.operators.operators2d.rst#fluidsim.operators.operators2d.OperatorsPseudoSpectral2D).

It contains the information on the grids (in physical and spectral space) and provides many optimized functions on arrays representing fields on these grids.

It has to be fast! For the two dimensional Fourier pseudo-spectral solvers, it is written in Cython.

## TimeStepping classes (`sim.time_stepping`)

`sim.time_stepping` is an instance of [fluidsim.base.time_stepping.pseudo_spect.TimeSteppingPseudoSpectral](../generated/fluidsim.base.time_stepping.pseudo_spect.rst#fluidsim.base.time_stepping.pseudo_spect.TimeSteppingPseudoSpectral), which is based on [fluidsim.base.time_stepping.base.TimeSteppingBase](../generated/fluidsim.base.time_stepping.base.rst#fluidsim.base.time_stepping.base.TimeSteppingBase).

This class contains the functions for the time advancement, i.e. Runge-Kutta functions and the actual loop than increments the time stepping index `sim.time_stepping.it`. The Runge-Kutta functions call the function `sim.tendencies_nonlin` and modify the state in Fourier space `sim.state.state_fft`.

The loop function also call the function `sim.output.one_time_step`.

## Output classes (`sim.output`)

`sim.output` is an instance of [fluidsim.solvers.ns2d.output.Output](../generated/fluidsim.solvers.ns2d.output.rst#fluidsim.solvers.ns2d.output.Output).

Saving and plotting of online or on-the-fly postprocessed data - i.e., data generated by processing the solver state variables at regular intervals during simulation time. It could include physical fields, spatially averaged means, spectral energy budgets, PDFs etc.

## Forcing classes (`sim.forcing`)

`sim.forcing` is an instance of [fluidsim.solvers.ns2d.forcing.ForcingNS2D](../generated/fluidsim.solvers.ns2d.forcing.rst#fluidsim.solvers.ns2d.forcing.ForcingNS2D).

If `params.forcing.enable` is True, it is used in `sim.tendencies_nonlin` to add the forcing term.